In [1]:
%%capture
import random 
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from numpy.linalg import *
from scipy.spatial import distance
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
np.random.seed(42)  # don't change this line

import base64
import datetime
!pip install recommenders

In [2]:
import sys

import itertools
import pandas as pd

from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.datasets.pandas_df_utils import filter_by
from recommenders.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var,
    map_at_k, ndcg_at_k, precision_at_k, recall_at_k
)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 16:07:37) 
[GCC 9.3.0]
Pandas version: 1.3.0


In [4]:
path = "https://5190-hav-recommendation-data.s3.us-east-1.amazonaws.com/ratings_Electronics.csv"
data = pd.read_csv(path, header=None)
data.columns = ['userId', 'productId', 'rating', 'timestamp']
data
#takes 10s or so

,userId,productId,rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
...,...,...,...,...
7824477,A2YZI3C9MOHC0L,BT008UKTMW,5.0,1396569600
7824478,A322MDK0M89RHN,BT008UKTMW,5.0,1313366400
7824479,A1MH90R0ADMIK0,BT008UKTMW,4.0,1404172800
7824480,A10M2KEFPEQDHN,BT008UKTMW,4.0,1297555200


In [5]:
user_rating_count = data['userId'].value_counts().rename('user_rating_count')
augmented_amazon_data = data.merge(user_rating_count.to_frame(), left_on='userId', right_index=True)
subset_df = augmented_amazon_data[augmented_amazon_data.user_rating_count >= 50]
print(subset_df.shape)
subset_df.head()
#10s

(125871, 5)


,userId,productId,rating,timestamp,user_rating_count
94,A3BY5KCNQZXV5U,0594451647,5.0,1390176000,50
14863,A3BY5KCNQZXV5U,B00000JD4V,4.0,1118016000,50
134213,A3BY5KCNQZXV5U,B000063574,5.0,1016668800,50
338368,A3BY5KCNQZXV5U,B0000CDJP8,5.0,1258761600,50
634048,A3BY5KCNQZXV5U,B0007Y794O,5.0,1369872000,50


In [6]:
product_rating_counts = subset_df['productId'].value_counts().rename('product_rating_counts')
product_rating_data   = subset_df.merge(product_rating_counts.to_frame(),
                                left_on='productId',
                                right_index=True)
product_rating_data = product_rating_data[product_rating_data.product_rating_counts >= 10]
product_rating_data.head()

,userId,productId,rating,timestamp,user_rating_count,product_rating_counts
634048,A3BY5KCNQZXV5U,B0007Y794O,5.0,1369872000,50,18
633970,AKT8TGIT6VVZ5,B0007Y794O,5.0,1147305600,192,18
633944,A1ILWPH1GHUXE2,B0007Y794O,4.0,1358553600,98,18
634073,A1ZM846Y7AUYD,B0007Y794O,4.0,1201132800,77,18
633998,A2ED50E3KWKUKW,B0007Y794O,5.0,1331856000,65,18


In [10]:
data=product_rating_data
data.drop(columns=['user_rating_count','product_rating_counts'],inplace=True)

In [11]:
# data = movielens.load_pandas_df(
#     size='100k',
#     header=['UserId', 'MovieId', 'Rating', 'Timestamp']
# )

#get rid of whitespace
data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
data.head()

,userId,productId,rating,timestamp
634048,A3BY5KCNQZXV5U,B0007Y794O,5.0,1369872000
633970,AKT8TGIT6VVZ5,B0007Y794O,5.0,1147305600
633944,A1ILWPH1GHUXE2,B0007Y794O,4.0,1358553600
634073,A1ZM846Y7AUYD,B0007Y794O,4.0,1201132800
633998,A2ED50E3KWKUKW,B0007Y794O,5.0,1331856000


In [12]:
rows_count, columns_count = data.shape
print('Total Number of rows :', rows_count)
print('Total Number of columns :', columns_count)
print(data.dtypes)

Total Number of rows : 42941
Total Number of columns : 4
userId        object
productId     object
rating       float64
timestamp      int64
dtype: object


In [13]:
unique_userId = data['userId'].nunique()
unique_productId = data['productId'].nunique()
print('Total number of unique Users    : ', unique_userId)
print('Total number of unique Products : ', unique_productId)

Total number of unique Users    :  1532
Total number of unique Products :  2263


In [14]:
data.isnull().sum()

userId       0
productId    0
rating       0
timestamp    0
dtype: int64

In [8]:
data.describe()

,rating,timestamp
count,7.824482e+06,7.824482e+06
mean,4.012337e+00,1.338178e+09
std,1.380910e+00,6.900426e+07
min,1.000000e+00,9.127296e+08
25%,3.000000e+00,1.315354e+09
50%,5.000000e+00,1.361059e+09
75%,5.000000e+00,1.386115e+09
max,5.000000e+00,1.406074e+09


In [9]:
data['rating'].value_counts()

5.0    4347541
4.0    1485781
1.0     901765
3.0     633073
2.0     456322
Name: rating, dtype: int64

In [10]:
data.shape
#7824482 rows, so grab the first 3912241 rows
#TEMPORAL DATASET SHIFT, ONLY UNCOMMENT WHEN DOING DATA SHIFT
#data=data.sort_values(by='timestamp')[0:3912241]

(7824482, 4)

In [15]:
train, test = python_random_split(data, ratio=0.70, seed=42)

In [16]:
#colnames
#userId       
#productId    
#rating       
#timestamp    

users_ratings = train.groupby(['userId'])['rating'].mean()
users_ratings = users_ratings.to_frame().reset_index()
users_ratings.rename(columns = {'rating': 'AvgRating'}, inplace = True)

users_ratings.head()
#takes about 25s

,userId,AvgRating
0,A100UD67AHFODS,4.777778
1,A100WO06OQR8BQ,3.368421
2,A105S56ODHGJEK,3.971429
3,A105TOJ6LTVMBG,3.888889
4,A10AFVU66A79Y1,4.400000


In [17]:
baseline_predictions = pd.merge(test, users_ratings, on=['userId'], how='inner')
#takes about 10s

,userId,productId,rating,timestamp,AvgRating


In [18]:
baseline_predictions = baseline_predictions[['userId', 'productId', 'AvgRating']]

cols = {
    'col_user': 'userId',
    'col_item': 'productId',
    'col_rating': 'rating',
    'col_prediction': 'AvgRating',
}

eval_rmse = rmse(test, baseline_predictions, **cols)
eval_mae = mae(test, baseline_predictions, **cols)
eval_rsquared = rsquared(test, baseline_predictions, **cols)
eval_exp_var = exp_var(test, baseline_predictions, **cols)

print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')


#takes about 40sec

RMSE:		0.947405
MAE:		0.697097
rsquared:	0.068852
exp var:	0.069041


In [20]:
item_counts = train['productId'].value_counts().to_frame().reset_index()
item_counts.columns = ['productId', 'Count']
item_counts.head()

,productId,Count
0,B0088CJT4U,148
1,B003ES5ZUU,124
2,B000N99BBC,119
3,B007WTAJTO,113
4,B00829TIEK,103


In [21]:
user_item_col = ['userId', 'productId']

# # Cross join users and items
test_users = test['userId'].unique()
user_item_list = list(itertools.product(test_users, item_counts['productId']))
users_items = pd.DataFrame(user_item_list, columns=user_item_col)

print("Number of user-item pairs:", len(users_items))

 # Remove seen items (items in the train set) as we will not recommend those again to the users
users_items_remove_seen = filter_by(users_items, train, user_item_col)

print("After remove seen items:", len(users_items_remove_seen))

#takes about 
#currently crashes ram..

Number of user-item pairs: 3374133
After remove seen items: 3344299


In [22]:
baseline_recommendations = pd.merge(item_counts, users_items_remove_seen, on=['productId'], how='inner')
baseline_recommendations.head()

,productId,Count,userId
0,B0088CJT4U,148,A28J3123I1QDKI
1,B0088CJT4U,148,AKX9EQ37PAYMY
2,B0088CJT4U,148,A2J8T58Z4X15IO
3,B0088CJT4U,148,A5KMMY627T3W
4,B0088CJT4U,148,A3B20R4ZAH0Y2Y


In [23]:
k = 10

cols['col_prediction'] = 'Count'

eval_map = map_at_k(test, baseline_recommendations, k=k, **cols)
eval_ndcg = ndcg_at_k(test, baseline_recommendations, k=k, **cols)
eval_precision = precision_at_k(test, baseline_recommendations, k=k, **cols)
eval_recall = recall_at_k(test, baseline_recommendations, k=k, **cols)

print("MAP:\t%f" % eval_map,
      "NDCG@K:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
#takes about 16s with filtered data

MAP:	0.012668
NDCG@K:	0.040214
Precision@K:	0.032059
Recall@K:	0.033992
